In [2]:
import csv
import requests
from bs4 import BeautifulSoup

def scrape_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        column_names = []

        # Find the table headers to get column names
        table = soup.find('table', {'class': 'results'})
        headers = table.find_all('th')
        column_names = [header.get_text(strip=True) for header in headers]

        # Find the table data and iterate through rows
        data = []
        for row in table.find_all('tr'):
            # Find table data cells
            cols = row.find_all('td')
            cols = [col.get_text(strip=True) for col in cols]
            data.append(cols)

        return column_names, data
    else:
        print(f"Error accessing the website. Status Code: {response.status_code}")
        return None, None
    

def save_to_csv(column_names, data, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(column_names)  # Write column names to CSV file
        writer.writerows(data)  # Write data to CSV file
    print(f"Data saved to {filename}")

if __name__ == "__main__":
    url = "https://xeno-canto.org/collection/species/all"  # Replace this with the actual URL

    column_names, scraped_data = scrape_data(url)
    if column_names and scraped_data:
        save_to_csv(column_names, scraped_data, "bird_species.csv")

Data saved to bird_species.csv


In [14]:
def extract_bird_songs(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        song_list = []
        
        if 'recordings' in data:
            for recording in data['recordings']:
                song_list.append(recording['en'])
        return song_list
    else:
        print(f"Failed to fetch data from {url}. Error code: {response.status_code}")
        return [] 
    
    
url = "https://xeno-canto.org/api/2/recordings?query=type:song"
bird_songs = extract_bird_songs(url)

def save_to_csv(data_list, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Bird Song Name'])
        for song_name in data_list:
            writer.writerow([song_name])
            
if bird_songs:
    csv_filename = "bird_songs.csv"
    save_to_csv(bird_songs, csv_filename)
    print(f"Data saved to {csv_filename}")
else:
    print("No bird songs data available.")
            


Data saved to bird_songs.csv
